# Steam to H2O3
This notebook provides a getting started tutorial for how to securely connect to an instance of the H2O AI Cloud from a local workstation and then accomplish common tasks using the H2O3.

##### For more H2O-3 tutorials for more data science details go to: 'https://github.com/h2oai/h2o-tutorials/blob/master/h2o-open-tour-2016/chicago/intro-to-h2o.ipynb'

## Notebook Setup
This tutorial relies on the latest Steam SDK which can be installed into a python environment using `pip install https://enterprise-steam.s3.amazonaws.com/release/1.8.9/python/h2osteam-1.8.9-py2.py3-none-any.whl`.

In [34]:
import h2osteam
import h2o
import os
import getpass
import h2o_mlops_client as mlops

from h2osteam.clients import H2oKubernetesClient

# Import H2O GLM:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

## Table of Contents
<div class="toc"><ul class="toc-item"><li><span><a href="#Notebook-Setup" data-toc-modified-id="Notebook-Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Notebook Setup</a></span></li><li><span><a href="#Securely-Connect" data-toc-modified-id="Securely-Connect-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Securely Connect</a></span></li><li><span><a href="#DAI-Instances" data-toc-modified-id="DAI-Instances-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Driverless AI Instances</a></span><ul class="toc-item"><li><span><a href="#Create-new-cluster" data-toc-modified-id="Create-new-cluster"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Create new cluster</a></span></li><li><span><a href="#list-all-existing-clusters" data-toc-modified-id="list-all-existing-clusters-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>List all existing clusters</a></span></li><li><span><a href="#add-a-dataset" data-toc-modified-id="add-a-dataset-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Add a dataset</a></span></li><li><span><a href="#Run-an-experiment" data-toc-modified-id="Run-an-experiment-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Run an experiment</a></span></li><li><span><a href="#Pause-our-instance" data-toc-modified-id="Pause-our-instance-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Pause our instance</a></span></li><li><span><a href="#Delete-the-instance" data-toc-modified-id="Delete-the-instance-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>Delete the instance</a></span></li></ul></li></ul></div>

## Securely Connect

To get the personal access token, login to the Steam you would like to test with, click on configuration and then token. Copy and paste the token into the box 

In [35]:
refresh_token = 'https://cloud-internal.h2o.ai/auth/get-platform-token'

In [36]:
print('Click link to get personalized password:', refresh_token)

tp = mlops.TokenProvider(
    token_endpoint_url = 'https://auth.demo.h2o.ai/auth/realms/q8s-internal/protocol/openid-connect/token',
    client_id = 'q8s-internal-platform',
    refresh_token=getpass.getpass()
)

Click link to get personalized password: https://cloud-internal.h2o.ai/auth/get-platform-token
········


In [37]:
steam = h2osteam.login(
    url="https://steam.cloud-internal.h2o.ai/",
    access_token=tp.ensure_fresh_token(),
)

## H2O3 Clusters

This example hows how to create an cluster and connect to it. First lets check the version of H2O python client and the available H2O server versions.

H2O Python Client Version:

In [38]:
h2o.__version__

'3.36.0.2'

H2O server version available:

In [39]:
h2osteam.api().get_h2o_engines()

[]

### Create new cluster

Now lets launch the cluster!

In [40]:
cluster = H2oKubernetesClient().launch_cluster(
    name="test_cluster",
    version="3.36.0.1",
)

To ensure the cluster is running, run the following line:

In [41]:
cluster.is_running()

True

#### Connecting to new cluster

Finally, lets connect to the cluster

In [42]:
cluster.connect()

Connecting to H2O server at https://steam.cloud-internal.h2o.ai:443/proxy/h2o-k8s/112 ... successful.


H2O_cluster_uptime:,3 mins 27 secs
H2O_cluster_timezone:,Etc/GMT
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.1
H2O_cluster_version_age:,2 months
H2O_cluster_name:,test_cluster
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.185 Gb
H2O_cluster_total_cores:,1
H2O_cluster_allowed_cores:,1
H2O_cluster_status:,"locked, healthy"


In [43]:
h2o.cluster().version

'3.36.0.1'

#### Connecting to existing clusters

If you want to connect to existing H2O cluster, run the following code!

In [44]:
name = "test_cluster"
cluster = H2oKubernetesClient.get_cluster(name)
cluster.connect()

/var/folders/hc/cb9t6hf90j9d04dxrvyk6t_c0000gp/T/ipykernel_21391/2545205769.py:2: UserWarning: 'H2oKubernetesClient.get_cluster()' function is deprecated in favor of 'H2oKubernetesClient().get_cluster()' and will be removed in v1.9
  cluster = H2oKubernetesClient.get_cluster(name)


Connecting to H2O server at https://steam.cloud-internal.h2o.ai:443/proxy/h2o-k8s/112 ... successful.


H2O_cluster_uptime:,3 mins 36 secs
H2O_cluster_timezone:,Etc/GMT
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.1
H2O_cluster_version_age:,2 months
H2O_cluster_name:,test_cluster
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.185 Gb
H2O_cluster_total_cores:,1
H2O_cluster_allowed_cores:,1
H2O_cluster_status:,"locked, healthy"


### List all existing clusters

In [45]:
clusters = H2oKubernetesClient().get_clusters()
clusters

### Add a dataset

In [48]:
#loan_csv = "/Volumes/H2OTOUR/loan.csv"  # modify this for your machine
# Alternatively, you can import the data directly from a URL
loan_csv = "https://raw.githubusercontent.com/h2oai/app-consumer-loan/master/data/loan.csv"

data = h2o.import_file(loan_csv)  # 163,987 rows x 15 columns

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [49]:
data.shape

(163987, 15)

In [52]:
h2o.cluster().show_status(True)

H2O_cluster_uptime:,14 mins 13 secs
H2O_cluster_timezone:,Etc/GMT
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.1
H2O_cluster_version_age:,2 months
H2O_cluster_name:,test_cluster
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.173 Gb
H2O_cluster_total_cores:,1
H2O_cluster_allowed_cores:,1
H2O_cluster_status:,"locked, healthy"


Nodes info:,Node 1
h2o,steam-1a3cbfec-5e5c-42be-a677-1958e0612c23-0/10.1.6.109:54321
healthy,True
last_ping,1646075391940.0000000
num_cpus,1
sys_load,0.88
mem_value_size,6048090
free_mem,3406542216.0000000
pojo_mem,68223262
swap_mem,0
free_disk,1084882944000.0000000


### Run an experiment
#### 1. First Split the dataset for training and testing

In [53]:
data['bad_loan'] = data['bad_loan'].asfactor()  #encode the binary repsonse as a factor
data['bad_loan'].levels()  #optional: after encoding, this shows the two factor levels, '0' and '1'

[['0', '1']]

In [54]:
splits = data.split_frame(ratios=[0.7, 0.15], seed=1)  

train = splits[0]
valid = splits[1]
test = splits[2]

In [55]:
y = 'bad_loan'
x = list(data.columns)

In [56]:
x.remove(y)  #remove the response
x.remove('int_rate')  #remove the interest rate column because it's correlated with the outcome

#### 2. Set up the experiment's settings (ie. accuracy, time, target column, etc.)

We first create an object of class, "H2OGeneralizedLinearEstimator". This does not actually do any training, it just sets the model up for training by specifying model parameters.

In [57]:
# Initialize the GLM estimator:
# Similar to R's glm() and H2O's R GLM, H2O's GLM has the "family" argument

glm_fit1 = H2OGeneralizedLinearEstimator(family='binomial', model_id='glm_fit1')

#### 3. Launch Experiment

Now that glm_fit1 object is initialized, we can train the model:

In [58]:
glm_fit1.train(x=x, y=y, training_frame=train, validation_frame=valid)

glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  glm_fit1


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,"Elastic Net (alpha = 0.5, lambda = 8.155E-5 )",82,52,5,py_5_sid_b772




ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.13984781035810745
RMSE: 0.37396231141400793
LogLoss: 0.44573426798537014
Null degrees of freedom: 114907
Residual degrees of freedom: 114855
Null deviance: 108939.63716428309
Residual deviance: 102436.86653132582
AIC: 102542.86653132582
AUC: 0.6741464380849886
AUCPR: 0.3129311477245476
Gini: 0.3482928761699773

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.1889005821903624: 


,,0,1,Error,Rate
0,0,62183.0,31841.0,0.3386,(31841.0/94024.0)
1,1,8549.0,12335.0,0.4094,(8549.0/20884.0)
2,Total,70732.0,44176.0,0.3515,(40390.0/114908.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...



Scoring History: 


,,timestamp,duration,iterations,negative_log_likelihood,objective,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_r2,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2022-02-28 19:11:50,0.000 sec,0,54469.818582,0.474030,,,,,,,,,,,,,,
1,,2022-02-28 19:11:51,0.504 sec,1,51474.873626,0.448341,,,,,,,,,,,,,,
2,,2022-02-28 19:11:51,0.587 sec,2,51227.197551,0.446199,,,,,,,,,,,,,,
3,,2022-02-28 19:11:51,0.649 sec,3,51218.459867,0.446130,,,,,,,,,,,,,,
4,,2022-02-28 19:11:51,0.712 sec,4,51218.457018,0.446130,,,,,,,,,,,,,,
5,,2022-02-28 19:11:51,0.937 sec,5,51218.433266,0.446130,0.373962,0.445734,0.059619,0.674146,0.312931,2.727179,0.351499,0.3768,0.450978,0.059927,0.675435,0.316372,2.641792,0.346518



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,purpose.small_business,0.664140,1.000000,0.077961
1,term.36 months,0.432335,0.650970,0.050750
2,purpose.credit_card,0.428948,0.645870,0.050353
3,annual_inc,0.401408,0.604402,0.047120
4,purpose.car,0.391287,0.589163,0.045932
5,addr_state.CO,0.315671,0.475307,0.037056
6,addr_state.TN,0.310776,0.467937,0.036481
7,term.60 months,0.302388,0.455308,0.035496
8,addr_state.DC,0.298106,0.448861,0.034994
9,addr_state.WV,0.296639,0.446652,0.034821



See the whole table with table.as_data_frame()


#### 4. View information, summary, model artifacts, and model performance of experiment

Let's see the performance of the GLM that were just trained. 

In [59]:
glm_perf1 = glm_fit1.model_performance(test)

Instead of printing the entire model performance metrics object, it is probably easier to print just the metric that you are interested in comparing. Here we are going to compare the test AUC to the training and validation AUC

In [60]:
print (glm_perf1.auc())

0.6774747329108557


In [61]:
print (glm_fit1.auc(train=True))
print (glm_fit1.auc(valid=True))

0.6741464380849886
0.6754349275583739


### Pause our instance

In [62]:
cluster.stop()

### Delete the instance

In [63]:
cluster.terminate()